In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo
import requests

/Users/mikenlee/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (2.0.3) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [21]:
#Configure chrome driver for clicking through pages o/w we could just use requests and get 
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False) #** unpacks values and opens browser

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/mikenlee/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [ ]:
# Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."


In [22]:
# results are returned as an iterable list
results = soup.find('li', class_="slide")
results

<li class="slide"><div class="image_and_description_container"><a href="/news/8822/a-martian-roundtrip-nasas-perseverance-rover-sample-tubes/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. </div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="A tray holding 39 sample tubes – each protected in a gold-colored sheath – is installed in NASA's Perseverance rover in this picture taken at the agency's Kennedy Space Center on May 21, 2020." src="/system/news_items/list_view_images/8822_1-PIA24304-CatScanMars-320x240.gif"/></div><div class="bottom_gradient"><div><h3>A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes</h3></div></div></a><div class="list_text"><div class="list_date">December 22, 2020</div><div c

In [68]:
def mars_news():
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)
    html = browser.html #html source code to parse
    soup = BeautifulSoup(html, "html.parser")
    article_container = soup.find('ul', class_='item_list') #scope down the soup object

    news_date = article_container.find("div", class_="list_date").text
    news_p = article_container.find("div", class_="article_teaser_body").text
    news_title = article_container.find("div", class_="content_title").find('a').text
    
    return news_date, news_p, news_title



('December 22, 2020',
 "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. ",
 "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes")

In [44]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html #html source code to parse
image_soup = BeautifulSoup(html, "html.parser")

In [82]:
def mars_img():
    
    base_url = 'https://www.jpl.nasa.gov'
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)
    html = browser.html #html source code to parse
    image_soup = BeautifulSoup(html, "html.parser")

    #METHOD 1: parsing through the style attribute in the article tag
    try:
        img_elem = image_soup.find('article', class_="carousel_item")['style']
        img_rel_url = img_elem.replace("background-image: url('", "")
        img_rel_url = img_rel_url.replace("');", "")
        print(img_rel_url) #relative URL to append to base URL

    except Exception as e:
        print(e)

    #METHOD 2
#     full_img_elem = browser.find_by_id('full_image')[0]
#     full_img_elem.click()

#     html = browser.html #html source code to parse for NEW PAGE
#     image_soup = BeautifulSoup(html, "html.parser")

#     img_rel_url = image_soup.find('img', class_ = "fancybox-image")['src']
#     print(img_rel_url)

    featured_img_url = f'{base_url}{img_rel_url2}'
    return featured_img_url

In [70]:
def mars_facts():
    url = "https://space-facts.com/mars/"
    browser.visit(url)

    mars_facts_df = pd.read_html(url)
    mars_facts_df = mars_facts_df[0] #pick first table
    mars_facts_df.columns = ['Description', 'Mars']
    mars_facts_html = mars_facts_df.to_html(border = 0, classes = 'table table-striped')

    return mars_facts_html

In [75]:
def scrape_all():
    #populate variables from functions
    news_date, news_p, news_title = mars_news()
    featured_img_url = mars_img()
    mars_facts_html = mars_facts()

    #assemble document to insert into database
    nasa_document = {
        'news_title': news_title,
        'news_paragraph': news_p,
        'featured_img_url': featured_img_url,
        'mars_facts_html': mars_facts_html
    }

    return nasa_document

In [65]:
base_url = "https://astrogeology.usgs.gov"
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

list_elem = [1,3,5,7]
hemisphere_image_urls = []

for link in list_elem:
    hemisphere_elem = browser.links.find_by_partial_href('map/Mars/Viking')[link]
    hemisphere_elem.click()

    html = browser.html
    image_soup = BeautifulSoup(html, "html.parser")

    img_rel_url = image_soup.find('img', class_ = "wide-image")['src']
    img_title = image_soup.find('h2', class_ = "title").text
    print(img_title)
    print(img_rel_url)
    hemisphere_image_urls.append({'title': img_title, 'img_url': f'{base_url}{img_rel_url}'})    

    url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(url)


Cerberus Hemisphere Enhanced
/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced
/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced
/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced
/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg


{'title': 'Valles Marineris Hemisphere Enhanced',
 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}

In [67]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [ ]:
#parse through using html tags and attributes
html = browser.html #html source code to parse for NEW PAGE
image_soup = BeautifulSoup(html, "html.parser")
#scope down the soup object
article_container = image_soup.find('ul', class_='item_list') 


img_rel_url = image_soup.find('img', class_ = "fancybox-image")['src']
print(img_rel_url)
hemisphere_url = f'{base_url}{img_rel_url}'

In [83]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#connect to mars DB; create if doesn't exist
db = client.mars_db

#connect to mars collection
mars = db.mars

#gather documents to insert
data_document = scrape_all()

#insert into mars collection
# mars.insert_one(data_document)

#upsert into database (prefered to avoid duplicates) 
mars.update_one({}, {'$set': data_document}, upsert = True)

/spaceimages/images/wallpaper/PIA17978-1920x1200.jpg
